In [1]:
from fastai.text import *   # Quick access to NLP functionality

### Merge positive and negative examplles for ruSentEval

In [2]:
data_path = Path('../data/ruSentEval/')

In [3]:
data_path.ls()

[PosixPath('../data/ruSentEval/texts.csv'),
 PosixPath('../data/ruSentEval/models'),
 PosixPath('../data/ruSentEval/tmp_clas'),
 PosixPath('../data/ruSentEval/texts-sample.csv'),
 PosixPath('../data/ruSentEval/tmp_ft_clas_sample'),
 PosixPath('../data/ruSentEval/tmp_lm'),
 PosixPath('../data/ruSentEval/tmp_clas_sample'),
 PosixPath('../data/ruSentEval/negative.csv'),
 PosixPath('../data/ruSentEval/tmp_lm_sample'),
 PosixPath('../data/ruSentEval/positive.csv')]

In [10]:
pos_df = pd.read_csv(data_path/'positive.csv', sep=';', header=None)

In [13]:
pos_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,positive
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,positive
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,positive
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,positive
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,positive


In [12]:
pos_df['label'] = 'positive'

In [17]:
pos_df.rename(columns={3:'text'}, inplace=True)

In [18]:
neg_df = pd.read_csv(data_path/'negative.csv', sep=';', header=None)

In [23]:
neg_df.head()

,0,1,2,text,4,5,6,7,8,9,10,11,label
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,negative
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,negative
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,negative
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0,negative
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0,negative


In [21]:
neg_df['label'] = 'negative'

In [22]:
neg_df.rename(columns={3:'text'}, inplace=True)

In [28]:
all_df = pd.concat([pos_df.loc[:, ['label', 'text']], neg_df.loc[:, ['label', 'text']]], ignore_index=True)

In [44]:
all_df['is_valid'] = np.random.choice([False, True], len(all_df), p=[0.9, 0.1])

In [45]:
all_df.head()

,label,text,is_valid
0,positive,"@first_timee хоть я и школота, но поверь, у на...",False
1,positive,"Да, все-таки он немного похож на него. Но мой ...",False
2,positive,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,False
3,positive,"RT @digger2912: ""Кто то в углу сидит и погибае...",False
4,positive,@irina_dyshkant Вот что значит страшилка :D\nН...,False


In [48]:
all_df.loc[all_df.label == 'positive', 'is_valid'].sum()

11529

In [49]:
all_df.loc[all_df.label == 'negative', 'is_valid'].sum()

11216

In [50]:
all_df.to_csv(data_path/'texts.csv', sep=',', index=False, header=True)

### Get 10% sample from merged data

In [3]:
data_path.ls()

[PosixPath('../data/ruSentEval/texts.csv'),
 PosixPath('../data/ruSentEval/models'),
 PosixPath('../data/ruSentEval/tmp_clas'),
 PosixPath('../data/ruSentEval/tmp_lm'),
 PosixPath('../data/ruSentEval/negative.csv'),
 PosixPath('../data/ruSentEval/positive.csv')]

In [4]:
all_df = pd.read_csv(data_path/'texts.csv')

In [5]:
all_df.head()

,label,text,is_valid
0,positive,"@first_timee хоть я и школота, но поверь, у на...",False
1,positive,"Да, все-таки он немного похож на него. Но мой ...",False
2,positive,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,False
3,positive,"RT @digger2912: ""Кто то в углу сидит и погибае...",False
4,positive,@irina_dyshkant Вот что значит страшилка :D\nН...,False


In [9]:
all_df.text.iloc[100:110].tolist()

['@jolechka3 получается,осторожно котлетку перекладываю в др.посуду)',
 '@redisskakat @girt_sk Единственное,что меня расстраивает на Бали. Невозможность сходить в кинотеатр с друзьями на интересный фильм :)',
 'Базарнов сегодня спрашивал - чем лучше сушиться :D Парень весит всего 60 кг где-то, ахаха. Но он сказал, что ему уже пора, лол',
 '@Zhopushonok маловато получается)))) лучше Незлобин тогда)',
 '@ekaetrina я люблю тебя со всеми твоими истериками, солнышко :D',
 'Урааа! @dolbysystem, вот!  я же говорила!))) #DavaiDinamo',
 'Все свое ношу с собой ) Хорошая штука все же #Dropbox. Кто еще не в теме - регистрируемся и получаем бонус ) https://t.co/WZ9LDhqnQO',
 'лента до сих пор не может угомониться с этим АЙ ЭМ\nголос похож на Билла, соглашусь, но бл, как так они написали сразу раста-песню? :D',
 'Дизайнер с большой буквы!) “@WoW_ReD: За 7 дней белые носки становятся черными, проходя "50 оттенков серого"”',
 '@ViktorVTsoy @dks_kamila Кама, совести у тебя нет, как ты не почувствовала 

In [12]:
sample = all_df.sample(frac=0.1)

In [14]:
sample.loc[sample.label == 'positive', 'is_valid'].sum()

1123

In [15]:
sample.loc[sample.label == 'negative', 'is_valid'].sum()

1103

In [18]:
len(sample.loc[sample.label == 'positive'])

11429

In [19]:
len(sample.loc[sample.label == 'negative'])

11254

In [20]:
sample.reset_index(drop=True, inplace=True)

In [22]:
sample.to_csv(data_path/'texts-sample.csv', sep=',', index=False, header=True)

### Merge ruSentEval and ruSentiment to use for wiki-LM finetining

In [ ]:
%%time

In [4]:
data_path.ls()

[PosixPath('../data/ruSentEval/texts.csv'),
 PosixPath('../data/ruSentEval/models'),
 PosixPath('../data/ruSentEval/tmp_clas'),
 PosixPath('../data/ruSentEval/texts-sample.csv'),
 PosixPath('../data/ruSentEval/tmp_ft_clas_sample'),
 PosixPath('../data/ruSentEval/tmp_lm'),
 PosixPath('../data/ruSentEval/tmp_clas_sample'),
 PosixPath('../data/ruSentEval/negative.csv'),
 PosixPath('../data/ruSentEval/tmp_lm_sample'),
 PosixPath('../data/ruSentEval/positive.csv')]

In [5]:
sent_eval = pd.read_csv(data_path/'texts.csv')

In [6]:
sent_eval.head()

,label,text,is_valid
0,positive,"@first_timee хоть я и школота, но поверь, у на...",False
1,positive,"Да, все-таки он немного похож на него. Но мой ...",False
2,positive,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,False
3,positive,"RT @digger2912: ""Кто то в углу сидит и погибае...",False
4,positive,@irina_dyshkant Вот что значит страшилка :D\nН...,False


In [7]:
sent_path = Path('../data/ruSentiment/')

In [8]:
sent_path.ls()

[PosixPath('../data/ruSentiment/models'),
 PosixPath('../data/ruSentiment/tmp_clas'),
 PosixPath('../data/ruSentiment/tmp_lm'),
 PosixPath('../data/ruSentiment/rusentiment_random_posts.csv'),
 PosixPath('../data/ruSentiment/rusentiment_test.csv'),
 PosixPath('../data/ruSentiment/rusentiment_preselected_posts.csv')]

In [9]:
sent = pd.read_csv(sent_path/'rusentiment_random_posts.csv')

In [11]:
sent.head()

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...


In [12]:
len(sent_eval), len(sent)

(226834, 21268)

In [14]:
merged_df = pd.concat([sent_eval.drop(columns='is_valid'), sent])

In [15]:
merged_df.reset_index(drop=True, inplace=True)

In [16]:
merged_df.head()

,label,text
0,positive,"@first_timee хоть я и школота, но поверь, у на..."
1,positive,"Да, все-таки он немного похож на него. Но мой ..."
2,positive,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,positive,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,positive,@irina_dyshkant Вот что значит страшилка :D\nН...


In [18]:
merged_df.index.nunique()

248102

In [19]:
merged_df.to_csv(sent_path/'merg_rusenteval_rusent.csv', sep=',', index=False, header=True)